<a href="https://colab.research.google.com/github/Cerino-rigo/EC3002C.602-2023/blob/main/Arbol_de_Decisiones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # **Modelos supervisados: Árbol de Decisión para Clasificación con `sklearn`</font>**


<p align="justify">
A continuación, se presentará el funcionamiento de los <b>árboles de decisión en problemas de clasificación</b>.
<br>
<br>
Los árboles de decisión en clasificación se utilizan para predecir variables respuesta categóricas. Los métodos de machine learning basados en árboles engloban a un conjunto de técnicas supervisadas no paramétricas que consiguen segmentar el espacio muestral en regiones más pequeñas.
<br>
<br>
La principal implementación de árboles de decisión en Python está disponible en la librería <code>scikit-learn</code> a través de las clases <code>DecisionTreeClassifier</code> y <code>DecisionTreeRegressor</code>.
<br>
<br>
Como criterio de selección de las divisiones óptimas existen varias alternativas, todas ellas con el objetivo de encontrar nodos lo más puros/homogéneos posible.
<br>
<br>
Las criterios para medir la pureza de los nodos más empleadas son:

- **Índice Gini**: cuantifica la varianza total en el conjunto de las  $n$ clases del nodo. Cuando Gini es 0, significa que ese nodo es totalmente puro. Por el contrario, si la frecuencia de cada clase es la misma, el valor del Índice Gini alcanza el valor máximo de 0.5. La impureza se refiere a cómo de mezcladas están las clases en cada nodo.
<br>
<br>  

$$G = 1- \sum_{i=1}^n ({p}_{i})^2$$
<br>

<p align="justify">
Donde $p_i$ representa la proporción de observaciones del nodo que pertenecen a
la clase  $i$.

- **Entropía**: es una forma de cuantificar el desorden de un sistema. En el caso de los nodos, el desorden se corresponde con la impureza. Si un nodo es puro, contiene únicamente observaciones de una clase, su entropía es 0. Por el contrario, si la frecuencia de cada clase es la misma, el valor de la entropía alcanza el valor máximo de 1.
<br>
<br>

$$ H = \sum_{i=1}^n -{p}_{i} \ log({p}_{i}) $$
<br>

Observa el siguiente ejemplo:

<p align="justify">
Graficamente la Entropía o pureza del nodo se puede visualizar del siguiente modo:


<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Arbol-002.png?raw=true" width="300">
</p>

## **Bibliotecas**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Operaciones matemáticas y estadísticas
import pandas as pd
import numpy as np

In [ ]:
# Visualización
import plotly.express as px
import plotly.graph_objs as go

## **Conjunto de Datos**

<p align="justify">
El conjunto de datos <code>Carseats</code>, contiene información sobre la venta de sillas infantiles en 400 tiendas distintas.
<br>
<br>
Para cada una de las 400 tiendas se han registrado 11 variables. Se pretende generar un modelo de clasificación que permita predecir si una tienda tiene ventas altas (<code>Sales</code> $>$ 9) o bajas (<code>Sales</code> $<=$ 9) en función de todas las variables disponibles.

In [ ]:
import statsmodels.api as sm
carseats = sm.datasets.get_rdataset("Carseats", "ISLR")
datos = carseats.data
print(carseats.__doc__)

In [ ]:
datos.head()

In [ ]:
datos.info()

<p align="justify">
Como <code>Sales</code> es una variable continua y el objetivo del estudio es clasificar las tiendas según si venden mucho o poco, se crea una nueva variable dicotómica (<code>'altas'</code>, <code>'bajas'</code>) llamada <code>sales</code>.

In [ ]:
datos['sales'] = datos.Sales.apply(lambda x: "altas" if x > 9 else "bajas")
#datos['sales'] = np.where(datos.Sales > 9, "altas", "bajas")

In [ ]:
datos.head()

<p align="justify">
Una vez creada la nueva variable respuesta categórica se descarta la original mediante el método <code>drop()</code>.

In [ ]:
datos = datos.drop(columns = 'Sales')

In [ ]:
datos.head()

## **Análisis Gráfico**

In [ ]:
px.scatter(datos,
           x='Price',
           y='Age',
           color='sales',
           template="gridon")

 ## **División del conjunto de datos</font>**

In [ ]:
X = datos.drop(columns=['sales'])
y = datos['sales']

 ## **Preprocesamiento de variables categóricas con `sklearn`</font>**

In [ ]:
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

<p align="justify">
Primero se identifica el nombre de las columnas categóricas y numéricas. El resultado es una <code>lista</code>.

In [ ]:
categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(X)

In [ ]:
categorical_columns

In [ ]:
#categorical_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()

In [ ]:
numerical_columns_selector = selector(dtype_include=[int,float])
numerical_columns = numerical_columns_selector(X)

In [ ]:
numerical_columns

<p align="justify">
Luego se aplica <b>one-hot-encoding</b> solo a las columnas categóricas. El parámetro <code>remainder</code> en <code>ColumnTransformer</code> determina cómo se deben manejar las columnas que no son seleccionadas o transformadas por los transformadores.
<br>
<br>
De forma predeterminada, el parámetro <code>remainder</code> está configurado en <code>drop</code>, lo que significa que cualquier columna restante que no esté especificada en los transformadores se eliminará.
<br>
<br>
Alternativamente, puedes establecer <code>remainder='passthrough'</code> para incluir las columnas restantes en la salida sin aplicar ninguna transformación. Esto es útil cuando se necesitan mantener ciertas columnas sin cambios, como en este caso las columnas numéricas.

In [ ]:
preprocessor = ColumnTransformer(
                    [('one-hot-encoding',
                      OneHotEncoder(handle_unknown='ignore',
                                    sparse_output=False),
                      categorical_columns)],
                    remainder='passthrough')

<p align="justify">
Una vez que se ha definido el objeto <code>ColumnTransformer</code>, con el método <code>fit_transform()</code> se aplican las tranformaciones al conjunto de datos <code>X</code>.

In [ ]:
X_encoded = preprocessor.fit_transform(X)
X_encoded

<p align="justify">
El resultado devuelto es un <code>numpy array</code>, por lo que se pierden los nombres de las columnas. Suele ser interesante poder inspeccionar cómo queda el conjunto de datos tras el preprocesado en formato <code>DataFrame</code>.
<br>
<br>
Por defecto, <code>OneHotEncoder</code> ordena las nuevas columnas de izquierda a derecha por orden alfabético.

Convertir el `numpy array` en `dataframe` y añadir el nombre de las columnas.

In [ ]:
columns_endoded = preprocessor.named_transformers_['one-hot-encoding'].get_feature_names_out(categorical_columns)
columns_endoded

In [ ]:
labels = np.concatenate([columns_endoded,numerical_columns])
X_transformed = pd.DataFrame(X_encoded, columns=labels)
X_transformed.head()

In [ ]:
X_transformed.info()

 ## **División del conjunto de entrenamiento y prueba</font>**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_transformed,
                                                    y,
                                                    random_state=123)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape



 ## **Ajuste y evaluación del modelo con `sklearn`</font>**

Documentación [Árbol de Decisión para Clasificación](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

<p align="justify">
En este ejemplo, se crea un de árbol de decisión para clasificación mediante la clase <code>DecisionTreeClassifier()</code> de <code>scikit_learn</code>. Luego, se ajusta el modelo a los datos de entrenamiento utilizando el método <code>fit()</code>, donde <code>X_train</code> representa las variabres predictoras de entrenamiento e </code>y_train</code> es la variable objetivo de entrenamiento.
<br>
<br>
Los hiperparámetros utilizados son:

- `max_depth = 2`: establece la profundidad máxima del árbol de decisión. Limita el número de niveles y nodos en el árbol. En este caso, el árbol tendrá una profundidad máxima de 2. Se estableció en este valor solo a fines pedagógicos para mostrar el funcionamiento del árbol.

- `criterion = 'entropy'`: especifica el criterio utilizado para medir la calidad de una división durante la construcción del árbol de decisión. El criterio de `'entropy'` utiliza el concepto de entropía de la información para medir la impureza de los nodos. Por defecto, el valor de este hiper-parámetro es `'gini'`.

- `random_state = 123`: establece la semilla aleatoria para garantizar la reproducibilidad. Al establecer el estado aleatorio en un valor específico (123 en este caso), el generador de números aleatorios producirá la misma secuencia de números aleatorios cada vez que ejecutes el código. Esto asegura que tus resultados sean consistentes y reproducibles.

In [ ]:
model_dt = DecisionTreeClassifier(max_depth = 2,
                                  criterion = 'entropy',
                                  random_state = 123
                                  )

In [ ]:
model_dt.fit(X_train, y_train)

<p align="justify">
Una vez ajustado el modelo, se pueden realizar predicciones en los datos de prueba utilizando el método <code>predict()</code> y evaluarlo con <code>score()</code>.

In [ ]:
prediction = model_dt.predict(X_test)
prediction

In [ ]:
y_test.values

In [ ]:
tabla = pd.DataFrame({"Prediccion":prediction,
                      "Real":y_test
                      })
tabla.head(15)

In [ ]:
model_dt.score(X_test, y_test)

<p align="justify">
La <b>exactitud</b> o <b>accuracy</b> del modelo es 0.72. Es decir, el modelo es capaz de predecir correctamente un 72 % de las observaciones del conjunto de prueba.

In [ ]:
from sklearn.model_selection import cross_validate
cv_results = cross_validate(model_dt, X_transformed, y, cv = 5)
cv_results

In [ ]:
scores = cv_results["test_score"]
print("")
print(f"La accuracy mediante cross-validation es: {scores.mean():.3f} ± {scores.std():.3f}")

 ## **Visualización del árbol de decisión</font>**

Documentación [Plot Tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html)

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 5))
print(f"Profundidad del árbol: {model_dt.get_depth()}")
print(f"Número de nodos terminales: {model_dt.get_n_leaves()}")

plot = plot_tree(
            decision_tree = model_dt,
            feature_names = labels.tolist(),
            class_names   = ['altas','bajas'],
            filled        = True,
            impurity      = True,
            fontsize      = 10,
            precision     = 2,
            ax            = ax
       )

<p align="justify">
El árbol de decisión generado se divide en función de las variables <code>ShelveLoc_Good</code> y <code>Price</code>. La raíz del árbol es la primera división que se realiza. A partir de ahí, cada nodo interno representa una pregunta o condición sobre las características y las ramas salientes representan las diferentes respuestas a esa pregunta.
<br>
<br>
El árbol de decisión se puede interpretar de la siguiente manera:

- En la raíz del árbol, se realiza la primera pregunta: "¿Es `ShelveLoc_Good` menor o igual a 0.5?"
 - Si es verdadero, se sigue por la rama izquierda, y el árbol realiza una segunda pregunta: "¿Es `Price` menor o igual a 74.5?"
    - Si es verdadero, se llega a una hoja del árbol donde se predice que las ventas son `altas`.
    - Si es falso, se llega a otra hoja del árbol donde se predice que las ventas son `bajas`.
 - Si es falso, se sigue por la rama derecha, y el árbol realizan una segunda pregunta: "¿Es `Price` menor o igual a 109.5?"
    - Si es verdadero, se llega a una hoja del árbol donde se predice que las ventas son `altas`.
    - Si es falso, se llega a otra hoja del árbol donde se predice que las ventas son `altas`.

<br>
<p align="justify">
Las hojas del árbol (nodos terminales) representan las predicciones finales para cada rama. En este caso, las hojas están etiquetadas como <code>altas</code> o <code>bajas</code>, según la clasificación prevista para cada combinación de variables.
<br>
<br>
Teniendo en cuenta el siguiente ejemplo se realiza la clasifición de ventas, suponga la sucursal en la posición 0 del conjunto de prueba que tiene las siguientes características:

In [ ]:
X_test.iloc[0,:]

<p align="justify">
Para esta sucursal <code>ShelveLoc_Good</code> es 1 (esto significa que el producto tiene una buena ubicación en la tienda), por lo tanto, mayor a 0.5, por lo tanto se sigue a la rama derecha y acá el precio es 129, el cual es mayor al umbral de 109.5, se sigue por la rama derecha y por lo tanto se predice que la venta en esa sucursal son <code>altas</code>.

In [ ]:
model_dt.predict(X_test)[0]

In [ ]:
y_test[0]

<p align="justify">
Los hiperparámetros <code>min_sample_leaf</code> y <code>min_samples_split</code> del algoritmo <code>DecisionTreeClassifier</code> permiten controlar la complejidad del árbol de decisión y evitar divisiones que puedan conducir a nodos con muy pocas muestras, lo que podría resultar en un <b>sobreajuste</b>.
<br>
<br>
Al aumentar los valores de estos hiperparámetros, se obtendrá un árbol de decisión más generalizado y menos complejo.


- `min_samples_leaf` especifica el número mínimo de muestras requeridas para que un nodo sea considerado una hoja (nodo terminal) en el árbol de decisión. Si el número de muestras en un nodo es menor que `min_samples_leaf`, no se realizará una división en ese nodo y se convertirá en una hoja. Este parámetro ayuda a evitar divisiones que produzcan hojas con muy pocas muestras, lo que puede ser útil para evitar sobreajuste. Garantiza un número mínimo de muestras en una hoja terminal.

- `min_samples_split` especifica el número mínimo de muestras requeridas para que se realice una división en un nodo. Si el número de muestras en un nodo es menor que `min_samples_split`, no se realizará ninguna división en ese nodo, y se convertirá en una hoja. Este parámetro controla la cantidad mínima de muestras necesarias para que un nodo sea elegible para realizar una división.

<br>
<p align="justify">
Comprobemos el efecto de incorporar el parámetro <code>min_samples_leaf</code>:

In [ ]:
model_dt_2 = DecisionTreeClassifier(min_samples_leaf = 10,
                                    random_state = 123
                                    ).fit(X_train, y_train)

In [ ]:
cv_results = cross_validate(model_dt_2, X_transformed, y)
cv_results

In [ ]:
scores = cv_results["test_score"]
print("")
print(f"La accuracy mediante cross-validation es: {scores.mean():.3f} ± {scores.std():.3f}")

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(18, 12))
print(f"Profundidad del árbol: {model_dt_2.get_depth()}")
print(f"Número de nodos terminales: {model_dt_2.get_n_leaves()}")

plot = plot_tree(
            decision_tree = model_dt_2,
            feature_names = labels.tolist(),
            class_names   = y,
            filled        = True,
            impurity      = True,
            fontsize      = 10,
            precision     = 2,
            ax            = ax
       )

 # **Conclusiones</font>**

<p align="justify">
A través de este ejercicio nosotros:
<br><br>
Utilizamos la biblioteca <code>scikit_learn</code> para entrenar un modelo de árbol de decisión en el contexto de un problema de clasificación.
<br>
Realizamos un análisis del comportamiento del árbol de decisión a través de la generación de un gráfico denominado <code>plot_tree</code>.
